In [12]:
import numpy as np
import pandas as pd
from pandas import Series
from pandas import DataFrame
import csv
import re

df = pd.read_csv("covid_19_set_3.csv")
df.head()

,ID,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
0,1,NaN,Afghanistan,33.93911,67.709953,2020/01/22,0,0,0,0,Eastern Mediterranean
1,2,NaN,Albania,41.1533,20.1683,2020/01/22,0,0,0,0,Europe
2,3,NaN,Algeria,28.0339,1.6596,2020/01/22,0,0,0,0,Africa
3,4,NaN,Andorra,42.5063,1.5218,2020/01/22,0,0,0,0,Europe
4,5,NaN,Angola,-11.2027,17.8739,2020/01/22,0,0,0,0,Africa


In [13]:
#Null entries

df.info() #polluted columns have 1000 null values each

#getting all rows polluted with null values
any_null = df[df.iloc[:, 2:].isnull().any(axis=1)] 
any_null #1000 rows, shows all rows with nulls are completely blank

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49068 entries, 0 to 49067
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              49068 non-null  int64 
 1   Province/State  14375 non-null  object
 2   Country/Region  48068 non-null  object
 3   Lat             48068 non-null  object
 4   Long            48068 non-null  object
 5   Date            48068 non-null  object
 6   Confirmed       48068 non-null  object
 7   Deaths          48068 non-null  object
 8   Recovered       48068 non-null  object
 9   Active          48068 non-null  object
 10  WHO Region      48068 non-null  object
dtypes: int64(1), object(10)
memory usage: 2.2+ MB


,ID,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
41,42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
113,114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179,180,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
48829,48830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48838,48839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48974,48975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49010,49011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
#Typos/outliers (Country/Region, WHO Region)

cr_counts = df['Country/Region'].value_counts()
print(cr_counts[cr_counts < 173]) #Country/Region values with a count < 172 are typos/outliers
wr_counts = df['WHO Region'].value_counts()
print(wr_counts[wr_counts < 2000]) #WHO Region values with a count < 1799 are typos/outliers

#getting polluted rows
polluted_cr_vals = cr_counts[cr_counts < 172].index
polluted_wr_vals = wr_counts[wr_counts < 1799].index
polluted_cr = df[df['Country/Region'].isin(polluted_cr_vals)]
polluted_wr = df[df['WHO Region'].isin(polluted_wr_vals)]

Country/Region
Armenia          172
Ch ina            17
Chnia             16
Chi na            13
Cina              13
                ... 
Netherland         1
Albnaia            1
Icel and           1
apan               1
Uited Kingdom      1
Name: count, Length: 753, dtype: int64
WHO Region
South-East Asia           1799
Euorpe                      27
Eruope                      24
Eurpoe                      23
Euroe                       23
                          ... 
Eastern Mediterranan         1
East ern Mediterranean       1
Eastern eMditerranean        1
Eastern Meditreranean        1
Eastern Me diterranean       1
Name: count, Length: 166, dtype: int64


In [15]:
#Wrong format (Date)

#checking year range
print(len(df[~df['Date'].str.match(r'^\s*2020\s*\S*\s*$', na = True)])) #dates outside of year 2020 are outliers

#getting polluted rows (entry is either not in yyyy/mm/dd format, not a real date, or not from year 2020)
polluted_da = df[~df['Date'].str.match(r'^\s*2020/(0[1-9]|1[0-2])/(0[1-9]|[1-2][0-9]|3[0-1])\s*$', na = True)]

#checking for not real dates not caught by above regex
print(len(df[df['Date'].str.match(r'^\s*2020/02/3[0-1]\s*$', na = False)])) 
print(len(df[df['Date'].str.match(r'^\s*2020/0(4|6|9)/31\s*$', na = False)]))
print(len(df[df['Date'].str.match(r'^\s*2020/11/31\s*$', na = False)]))

431
0
0
0


In [16]:
#(Confirmed, Deaths, Recovered, Active)

#Value is not a non-negative integer
polluted_c = df[~df['Confirmed'].astype("string").str.isdecimal()]
polluted_de = df[~df['Deaths'].astype("string").str.isdecimal()]
polluted_r = df[~df['Recovered'].astype("string").str.isdecimal()]
polluted_a = df[~df['Active'].astype("string").str.isdecimal()]

#Values are non-negative integers, but Active != Confirmed - Deaths - Recovered
#Note: will be listed as pollution in the 'Active' column
cdera_nnints = df[df['Active'].astype("string").str.isdecimal() & df['Confirmed'].astype("string").str.isdecimal() & df['Deaths'].astype("string").str.isdecimal() & df['Recovered'].astype("string").str.isdecimal()]
polluted_a2 = df[df['ID'].isin(cdera_nnints['ID']) & (pd.to_numeric(df['Active'], errors='coerce') != pd.to_numeric(df['Confirmed'], errors='coerce') - pd.to_numeric(df['Deaths'], errors='coerce') - pd.to_numeric(df['Recovered'], errors='coerce'))]

#Outliers 
#ISSUE: might disproportionately affect entries from regions with large populations
#getting rows with non-negative ints for each column
c_nnints = df[df['Confirmed'].astype("string").str.isdecimal()]
de_nnints = df[df['Deaths'].astype("string").str.isdecimal()]
r_nnints = df[df['Recovered'].astype("string").str.isdecimal()]
a_nnints = df[df['Active'].astype("string").str.isdecimal()]

#finding outliers (value is <= Q1 - 1.5IQR or >= Q3 + 1.5IQR)
#note: only looks at rows with non-negative ints, so no need to specify that lower bound should be >= 0
c_q1 = c_nnints['Confirmed'].astype(int).quantile(0.25)
c_q3 = c_nnints['Confirmed'].astype(int).quantile(0.75)
c_iqr = c_q3 - c_q1
print('Confirmed: Outlier if <=', c_q1 - 1.5*c_iqr, 'or >=', c_q3 + 1.5*c_iqr)
c_outliers = c_nnints[(c_nnints['Confirmed'].astype(int) <= (c_q1 - 1.5*c_iqr)) | (c_nnints['Confirmed'].astype(int) >= (c_q3 + 1.5*c_iqr))]
print(len(c_outliers))

de_q1 = de_nnints['Deaths'].astype(int).quantile(0.25)
de_q3 = de_nnints['Deaths'].astype(int).quantile(0.75)
de_iqr = de_q3 - de_q1
print('Deaths: Outlier if <=', de_q1 - 1.5*de_iqr, 'or >=', de_q3 + 1.5*de_iqr)
de_outliers = de_nnints[(de_nnints['Deaths'].astype(int) <= (de_q1 - 1.5*de_iqr)) | (de_nnints['Deaths'].astype(int) >= (de_q3 + 1.5*de_iqr))]
print(len(de_outliers))

r_q1 = r_nnints['Recovered'].astype(int).quantile(0.25)
r_q3 = r_nnints['Recovered'].astype(int).quantile(0.75)
r_iqr = r_q3 - r_q1
print('Recovered: Outlier if <=', r_q1 - 1.5*r_iqr, 'or >=', r_q3 + 1.5*r_iqr)
r_outliers = r_nnints[(r_nnints['Recovered'].astype(int) <= (r_q1 - 1.5*r_iqr)) | (r_nnints['Recovered'].astype(int) >= (r_q3 + 1.5*r_iqr))]
print(len(r_outliers))

a_q1 = a_nnints['Active'].astype(int).quantile(0.25)
a_q3 = a_nnints['Active'].astype(int).quantile(0.75)
a_iqr = a_q3 - a_q1
print('Active: Outlier if <=', a_q1 - 1.5*a_iqr, 'or >=', a_q3 + 1.5*a_iqr)
a_outliers = a_nnints[(a_nnints['Active'].astype(int) <= (a_q1 - 1.5*a_iqr)) | (a_nnints['Active'].astype(int) >= (a_q3 + 1.5*a_iqr))]
#remove rows where Active != Confirmed - Deaths - Recovered (already accounted for)
a_outliers = a_outliers[~a_outliers['ID'].isin(polluted_a2['ID'])]
print(len(a_outliers))



Confirmed: Outlier if <= -2298.5 or >= 3841.5
8511
Deaths: Outlier if <= -48.0 or >= 80.0
8733
Recovered: Outlier if <= -1059.0 or >= 1765.0
7707
Active: Outlier if <= -937.5 or >= 1562.5
8351


In [17]:
#(Lat, Long)

#Not a number 
#Not in valid range (-90 to 90 for lat, -180 to 180 for long)
#Not 4 decimal places (as most entries seem to have that amount)
polluted_la = df[~df['Lat'].str.match(r'^\s*-?([1-8]?[0-9]|90).\d{4}\s*$', na = True)]
polluted_lo = df[~df['Long'].str.match(r'^\s*-?([1-9]?[0-9]|1[0-7][0-9]|180).\d{4}\s*$', na = True)]

#ISSUE: 4 decimal place requirement might be too strict?
print(len(polluted_la))
print(len(polluted_lo))

15919
15752


In [18]:
with open('jqy2_polluted_set_3.csv', 'w', newline = '') as newcsvfile:
    writer = csv.writer(newcsvfile)
    writer.writerow(['ID', 'Column Name', 'Justification'])
    for row in any_null['ID']:
        writer.writerow([row, '', 'Null (entire row)']) 
    for row in polluted_cr['ID']:
        writer.writerow([row, 'Country/Region', 'Typo or outlier'])
    for row in polluted_wr['ID']:
        writer.writerow([row, 'WHO Region', 'Typo or outlier'])
    for row in polluted_da['ID']:
        writer.writerow([row, 'Date', 'Not yyyy/mm/dd, fake date, or year != 2020 (outlier)'])
    for row in polluted_c['ID']:
        writer.writerow([row, 'Confirmed', 'Not a non-negative int'])
    for row in polluted_de['ID']:
        writer.writerow([row, 'Deaths', 'Not a non-negative int'])
    for row in polluted_r['ID']:
        writer.writerow([row, 'Recovered', 'Not a non-negative int'])
    for row in polluted_a['ID']:
        writer.writerow([row, 'Active', 'Not a non-negative int'])
    for row in polluted_a2['ID']:
        writer.writerow([row, 'Active', '!= Confirmed - Deaths - Recovered'])
    for row in c_outliers['ID']:
        writer.writerow([row, 'Confirmed', 'Outlier (IQR method)'])
    for row in de_outliers['ID']:
        writer.writerow([row, 'Deaths', 'Outlier (IQR method)'])
    for row in r_outliers['ID']:
        writer.writerow([row, 'Recovered', 'Outlier (IQR method)'])
    for row in a_outliers['ID']:
        writer.writerow([row, 'Active', 'Outlier (IQR method)'])
    for row in polluted_la['ID']:
        writer.writerow([row, 'Lat', 'Not number from -90 to 90 with 4 decimal places'])
    for row in polluted_lo['ID']:
        writer.writerow([row, 'Long', 'Not number from -180 to 180 with 4 decimal places'])